<a href="https://colab.research.google.com/github/MaxTTG/Cpp-CUDA/blob/main/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-x_ph2j1q
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-x_ph2j1q
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=a77c7a2f48aa9913469d62e93121cc725fb6f27f0273645845a946f2f37e1790
  Stored in directory: /tmp/pip-ephem-wheel-cache-cpb3ze2y/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
!/usr/local/cuda/bin/nvcc --version
#!pip install --upgrade git+https://github.com/frehseg/nvcc4jupyter.git
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples/

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pdw_d8lv
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pdw_d8lv
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
Copying samples to /root/NVIDIA_CUDA-11.2_Samples now...
Finished copying samples.


Тестовый запуск C++

In [ ]:
%%cu
#include <iostream>
  int main()
{
  std::cout << "Hello world!!!";
  return 0;
}

Hello world!!!


CUDA сложение векторов

In [ ]:
%%cu

#include <cublas_v2.h>
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>

__global__ void kernel(double* a, double* b, double* c, unsigned int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  for(;i < n; i += blockDim.x * gridDim.x)
  {
      c[i] = a[i] + b[i];
  }
}

int main(int argc, char* argv[])
{
    int n = 1100000;
    int n2b = n * sizeof(double);
    int n2 = n;
    printf("double\n");
    printf("n = %d\n", n);
    printf("n2b = %d\n", n2b);
    printf("n2 = %d\n", n2);

    // Выделение памяти на хосте
    double* a = (double*)calloc(n2, sizeof(double));
    double* b = (double*)calloc(n2, sizeof(double));
    double* c = (double*)calloc(n2, sizeof(double));

    for (int i = 0; i < n; ++i) {
        a[i] = 1;
        b[i] = 2;
    }
    printf("a[0] = %f\n", a[0]);
    printf("b[0] = %f\n", b[0]);
    // Выделение памяти на устройстве
    double* adev = NULL;
    double* bdev = NULL;
    double* cdev = NULL;

    cudaMalloc((void**)&adev, n2b);
    cudaMalloc((void**)&bdev, n2b);
    cudaMalloc((void**)&cdev, n2b);

    // Создание обработчиков событий
    cudaEvent_t start, stop;
    float gpuTime = 0.0f;
    cudaEventCreate(&start);

    cudaEventCreate(&stop);

    // Копирование данных с хоста на девайс
    cudaMemcpy(adev, a, n2b, cudaMemcpyHostToDevice);
    cudaMemcpy(bdev, b, n2b, cudaMemcpyHostToDevice);

    // Установка точки старта
    cudaEventRecord(start, 0);

    //Запуск ядра
    //[(4096,1024);(2048,1024);(4096,256);(2048,256);(4096,64);(2048,64)]
    int q = 4096;
    int p = 1024;
    printf("GridDim = %d, BlockDim = %d\n", q, p);

    kernel <<< q, p >>> (adev, bdev, cdev, n);

    cudaGetLastError();

    // Синхронизация устройств
    cudaDeviceSynchronize();

    // Установка точки окончания
    cudaEventRecord(stop, 0);

    // Копирование результата на хост
    cudaMemcpy(c, cdev, n2b, cudaMemcpyDeviceToHost);

    // Расчет времени
    cudaEventElapsedTime(&gpuTime, start, stop);

    printf("time spent executing %s: %.9f seconds\n", "kernel", gpuTime / 1000);
    for (int i = 0; i < 5; ++i) {
        printf("i = %d) a: %f b: %f c: %f\n", i, a[i], b[i], c[i]);
    }
    for (int i = n - 6; i < n; ++i) {
        printf("i = %d) a: %f b: %f c: %f\n", i, a[i], b[i], c[i]);
    }

    for (int i = 0; i < n; ++i) {
        if (c[i] == 0) {
            printf("i = %d; c[i] = 0", i);
            break;
        }
    }
    // Очищение памяти
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(adev);
    cudaFree(bdev);
    cudaFree(cdev);
    free(a);
    free(b);
    free(c);
    return 0;
}

double
n = 1100000
n2b = 8800000
n2 = 1100000
a[0] = 1.000000
b[0] = 2.000000
GridDim = 4096, BlockDim = 1024
time spent executing kernel: 0.000156192 seconds
i = 0) a: 1.000000 b: 2.000000 c: 3.000000
i = 1) a: 1.000000 b: 2.000000 c: 3.000000
i = 2) a: 1.000000 b: 2.000000 c: 3.000000
i = 3) a: 1.000000 b: 2.000000 c: 3.000000
i = 4) a: 1.000000 b: 2.000000 c: 3.000000
i = 1099994) a: 1.000000 b: 2.000000 c: 3.000000
i = 1099995) a: 1.000000 b: 2.000000 c: 3.000000
i = 1099996) a: 1.000000 b: 2.000000 c: 3.000000
i = 1099997) a: 1.000000 b: 2.000000 c: 3.000000
i = 1099998) a: 1.000000 b: 2.000000 c: 3.000000
i = 1099999) a: 1.000000 b: 2.000000 c: 3.000000



Последовательный вариант сложения векторов

In [ ]:
%%cu

#include <stdio.h>
#include <ctime>
#include <stdlib.h>

int main()
{
    int n = 9900000;
    double* a = (double*)calloc(n, sizeof(double));
    double* b = (double*)calloc(n, sizeof(double));
    double* c = (double*)calloc(n, sizeof(double));
    for (int i = 0; i < n; ++i)
    {
        a[i] = 1;
        b[i] = 2;
    }
    int start = clock();

    for (int i = 0; i < n; ++i)
        c[i] = a[i] + b[i];

    int end = clock();
    printf("Total time: %f \n", (float)(end - start) / 1000000);
    for (int i = 0; i < 5; ++i) {
        printf("i = %d) a: %f b: %f c: %f\n", i, a[i], b[i], c[i]);
    }
    for (int i = n - 6; i < n; ++i) {
        printf("i = %d) a: %f b: %f c: %f\n", i, a[i], b[i], c[i]);
    }
}

Total time: 0.034360 
i = 0) a: 1.000000 b: 2.000000 c: 3.000000
i = 1) a: 1.000000 b: 2.000000 c: 3.000000
i = 2) a: 1.000000 b: 2.000000 c: 3.000000
i = 3) a: 1.000000 b: 2.000000 c: 3.000000
i = 4) a: 1.000000 b: 2.000000 c: 3.000000
i = 9899994) a: 1.000000 b: 2.000000 c: 3.000000
i = 9899995) a: 1.000000 b: 2.000000 c: 3.000000
i = 9899996) a: 1.000000 b: 2.000000 c: 3.000000
i = 9899997) a: 1.000000 b: 2.000000 c: 3.000000
i = 9899998) a: 1.000000 b: 2.000000 c: 3.000000
i = 9899999) a: 1.000000 b: 2.000000 c: 3.000000



CUDA умножение матриц

In [ ]:
%%cuda --name my_curand.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cstdlib>
#include <malloc.h>
#include <curand.h>

// GPU_fill_rand() - Функция случайной генерации матрицы
// Заполнить массив A(nr_rows_A, nr_cols_A) случайными числами на графическом процессоре
void GPU_fill_rand(float* A, int nr_rows_A, int nr_cols_A) {
    // Создание генератора псевдо-рандомных чисел
    curandGenerator_t prng;
    curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_DEFAULT);

    // Установка сида для генератора
    curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long) clock());

    // Заполнение матрицы случайными числами
    curandGenerateUniform(prng, A, nr_rows_A * nr_cols_A);
}

// gpu_blas_mmul() - Функция умножения матриц
// Перемножение матриц А и В с сохранением результата в матрице С
// C(m,n) = A(m,k) * B(k,n)
void gpu_blas_mmul(const float* A, const float* B, float* C, const int m, const int k, const int n) {
    int lda = m, ldb = k, ldc = m;
    const float alf = 1;
    const float bet = 0;
    const float* alpha = &alf;
    const float* beta = &bet;
    // Create a handle for CUBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);
    // Do the actual multiplication
    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);
    // Destroy the handle
    cublasDestroy(handle);
}

//print_matrix() - Функция вывода матрицы
void print_matrix(float* matrix, int rows, int cols) {

    printf("%f\t*\t*\t*\t%f\n", matrix[0], matrix[rows * (rows - 1)]);
    printf("*\t*\t*\t*\t*\t*\n");
    printf("%f\t*\t*\t*\t%f\n", matrix[rows], matrix[cols * rows - 1]);
    printf("\n");

}

int main() {
    // Выделяем 3 массива на хосте
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
    cudaEvent_t start, end;
    float time = 0.0f;
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    // Используются квадратные матрицы
    nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = nr_rows_C = nr_cols_C = 1260;
    //140 420 1260

    printf("float\nn = %d\n\n", nr_rows_A);

    float* h_A = (float*)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float* h_B = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));
    float* h_C = (float*)malloc(nr_rows_C * nr_cols_C * sizeof(float));

    // Выделяем 3 массива на девайсе
    float* d_A, * d_B, * d_C;
    cudaMalloc(&d_A, nr_rows_A * nr_cols_A * sizeof(float));
    cudaMalloc(&d_B, nr_rows_B * nr_cols_B * sizeof(float));
    cudaMalloc(&d_C, nr_rows_C * nr_cols_C * sizeof(float));

    // Заполняем матрицы А и В случайными числами
    GPU_fill_rand(d_A, nr_rows_A, nr_cols_A);
    GPU_fill_rand(d_B, nr_rows_B, nr_cols_B);

    // Можно скопировать их на хост, чтобы вывести их
    cudaMemcpy(h_A, d_A, nr_rows_A * nr_cols_A * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_B, d_B, nr_rows_B * nr_cols_B * sizeof(float), cudaMemcpyDeviceToHost);
    printf("A =\n");
    print_matrix(h_A, nr_rows_A, nr_cols_A);
    printf("B =\n");
    print_matrix(h_B, nr_rows_B, nr_cols_B);


    cudaEventRecord(start, 0);
    // Умножаем матрицы А и В на GPU
    gpu_blas_mmul(d_A, d_B, d_C, nr_rows_A, nr_cols_A, nr_cols_B);
    cudaEventRecord(end, 0);
    // Копируем на хост и выводим результат умножения матриц
    cudaMemcpy(h_C, d_C, nr_rows_C * nr_cols_C * sizeof(float), cudaMemcpyDeviceToHost);
    printf("C =\n");
    print_matrix(h_C, nr_rows_C, nr_cols_C);

    cudaEventElapsedTime(&time, start, end);
    printf("Total time: %f\n", time / 1000);

    // Освобождаем память на девайсе
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Освобождаем память на хосте
    free(h_A);
    free(h_B);
    free(h_C);
    return 0;
}


'File written in /content/src/my_curand.cu'

In [ ]:
!nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand -lcublas

In [ ]:
!/content/src/my_curand

float
n = 1260

A =
0.220347	*	*	*	0.696221
*	*	*	*	*	*
0.002065	*	*	*	0.792222

B =
0.237537	*	*	*	0.440715
*	*	*	*	*	*
0.091920	*	*	*	0.902945

C =
314.827698	*	*	*	330.303467
*	*	*	*	*	*
307.319611	*	*	*	325.829498

Total time: 0.543147


Генерация матрицы на GPU

In [ ]:
%%cuda --name rand_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cstdlib>
#include <malloc.h>
#include <curand.h>

// GPU_fill_rand() - Функция случайной генерации матрицы
// Заполнить массив A(nr_rows_A, nr_cols_A) случайными числами на графическом процессоре
void GPU_fill_rand(float* A, int nr_rows_A, int nr_cols_A) {
    // Создание генератора псевдо-рандомных чисел
    curandGenerator_t prng;
    curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_DEFAULT);

    // Установка сида для генератора
    curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long) clock());

    // Заполнение матрицы случайными числами
    curandGenerateUniform(prng, A, nr_rows_A * nr_cols_A);
}
//print_matrix() - Функция вывода матрицы
void print_matrix(float* matrix, int rows, int cols) {

    printf("%f\t*\t*\t*\t%f\n", matrix[0], matrix[rows * (rows - 1)]);
    printf("*\t*\t*\t*\t*\t*\n");
    printf("%f\t*\t*\t*\t%f\n", matrix[rows], matrix[cols * rows - 1]);
    printf("\n");

}

int main() {
    // Выделяем 3 массива на хосте
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
    cudaEvent_t start, end;
    float time = 0.0f;
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    // Используются квадратные матрицы
    nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = nr_rows_C = nr_cols_C = 420;
    //140 420 1260

    printf("float\nn = %d\n\n", nr_rows_A);

    float* h_A = (float*)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float* h_B = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));
    float* h_C = (float*)malloc(nr_rows_C * nr_cols_C * sizeof(float));

    // Выделяем 3 массива на девайсе
    float* d_A, * d_B, * d_C;
    cudaMalloc(&d_A, nr_rows_A * nr_cols_A * sizeof(float));
    cudaMalloc(&d_B, nr_rows_B * nr_cols_B * sizeof(float));
    cudaMalloc(&d_C, nr_rows_C * nr_cols_C * sizeof(float));


    cudaEventRecord(start, 0);

    // Заполняем матрицы А и В случайными числами
    GPU_fill_rand(d_A, nr_rows_A, nr_cols_A);
    GPU_fill_rand(d_B, nr_rows_B, nr_cols_B);

    cudaEventRecord(end, 0);

    // Можно скопировать их на хост, чтобы вывести их
    cudaMemcpy(h_A, d_A, nr_rows_A * nr_cols_A * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_B, d_B, nr_rows_B * nr_cols_B * sizeof(float), cudaMemcpyDeviceToHost);
    printf("A =\n");
    print_matrix(h_A, nr_rows_A, nr_cols_A);
    printf("B =\n");
    print_matrix(h_B, nr_rows_B, nr_cols_B);

    cudaEventElapsedTime(&time, start, end);
    printf("GPU generate\nn = %d\nTotal time: %f\n", nr_rows_A, time / 1000);

    // Освобождаем память на девайсе
    cudaFree(d_A);
    cudaFree(d_B);

    // Освобождаем память на хосте
    free(h_A);
    free(h_B);
    return 0;
}

'File written in /content/src/rand_gpu.cu'

In [ ]:
!nvcc -o /content/src/rand_gpu /content/src/rand_gpu.cu -lcurand -lcublas

In [ ]:
!/content/src/rand_gpu

float
n = 420

A =
0.697885	*	*	*	0.705352
*	*	*	*	*	*
0.187291	*	*	*	0.693522

B =
0.988364	*	*	*	0.259889
*	*	*	*	*	*
0.319250	*	*	*	0.551665

GPU generate
n = 420
Total time: 0.008935


Генерация на CPU

In [ ]:
%%cu

#include <stdio.h>
#include <ctime>

void print_matrix(float* matrix, int rows, int cols) {
    printf("%f\t*\t*\t*\t%f\n", matrix[0], matrix[rows * (rows - 1)]);
    printf("*\t*\t*\t*\t*\t*\n");
    printf("%f\t*\t*\t*\t%f\n", matrix[rows], matrix[cols * rows - 1]);
    printf("\n");
}

int main() {
    // Выделяем 3 массива на хосте
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B;

    // Используются квадратные матрицы
    nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = 140;
    //140 420 1260

    printf("float\nn = %d\n\n", nr_rows_A);

    float* h_A = (float*)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float* h_B = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));

    srand(time(NULL));
    unsigned int start =  clock(); // начальное время
    for (int i = 0; i < nr_rows_A * nr_cols_A; ++i)
        h_A[i] = (double)(rand())/RAND_MAX;

    for (int i = 0; i < nr_rows_B * nr_cols_B; ++i)
        h_B[i] = (double)(rand())/RAND_MAX;

    unsigned int end = clock(); // конечное время
    unsigned int time = end - start; // искомое время
    printf("A =\n");
    print_matrix(h_A, nr_rows_A, nr_cols_A);
    printf("B =\n");
    print_matrix(h_B, nr_rows_B, nr_cols_B);
    printf("СPU generate\nTotal time: %f\n", (double)time / 1000000);
    return 0;
}

float
n = 140

A =
0.449748	*	*	*	0.139235
*	*	*	*	*	*
0.424846	*	*	*	0.321787

B =
0.273017	*	*	*	0.250305
*	*	*	*	*	*
0.238729	*	*	*	0.195733

СPU generate
Total time: 0.000387



Умножение матриц на CPU

In [ ]:
%%cu

#include <stdio.h>
#include <ctime>

void print_matrix(float* matrix, int rows, int cols) {
    printf("%f\t*\t*\t*\t%f\n", matrix[0], matrix[rows * (rows - 1)]);
    printf("*\t*\t*\t*\t*\t*\n");
    printf("%f\t*\t*\t*\t%f\n", matrix[rows], matrix[cols * rows - 1]);
    printf("\n");
}

void mult_cpu(float* A, float* B, float* C, int nA, int nB) {
    for (int i = 0; i < nA; ++i) {
        for (int j = 0; j < nB; ++j) {
            for (int k = 0; k < nB; ++k)
                C[i * nB + j] += A[i * nB + k] * B[k * nB + j];
        }
    }
}

int main() {
    // Выделяем 3 массива на хосте
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B;

    // Используются квадратные матрицы
    nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = 1260;
    //140 420 1260

    float* h_A = (float*)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float* h_B = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));
    float* C = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));

    srand(time(NULL));
    for (int i = 0; i < nr_rows_A * nr_cols_A; ++i)
        h_A[i] = (double)(rand())/RAND_MAX;

    for (int i = 0; i < nr_rows_B * nr_cols_B; ++i)
        h_B[i] = (double)(rand())/RAND_MAX;

    for (int i = 0; i < nr_rows_B * nr_cols_B; ++i)
        C[i] = 0;

    unsigned int start =  clock(); // начальное время

    mult_cpu(h_A, h_B, C, nr_rows_A, nr_rows_B);

    unsigned int end = clock(); // конечное время
    unsigned int time = end - start; // искомое время
    printf("A =\n");
    print_matrix(h_A, nr_rows_A, nr_cols_A);
    printf("B =\n");
    print_matrix(h_B, nr_rows_B, nr_cols_B);
    printf("C =\n");
    print_matrix(C, nr_rows_B, nr_cols_B);
    printf("СPU generate\nTotal time: %f\n", (double)time / 1000000);
    free(h_A);
    free(h_B);
    free(C);
    return 0;
}

A =
0.916938	*	*	*	0.173154
*	*	*	*	*	*
0.246181	*	*	*	0.244513

B =
0.515437	*	*	*	0.871140
*	*	*	*	*	*
0.962112	*	*	*	0.437043

C =
321.252716	*	*	*	317.184631
*	*	*	*	*	*
317.543274	*	*	*	317.640015

СPU generate
Total time: 14.156591

